In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/sklearn/topic_modelling/imdb')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install pyldavis

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf

In [4]:
def get_idx2word(_index_from=3):
  word2idx = tf.keras.datasets.imdb.get_word_index()
  word2idx = {k:(v+_index_from) for k,v in word2idx.items()}
  word2idx["<pad>"] = 0
  word2idx["<start>"] = 1
  word2idx["<unk>"] = 2
  idx2word = {idx: w for w, idx in word2idx.items()}
  return idx2word

In [5]:
N_TOPICS = 10
MAX_TERMS = 10

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data()
idx2word = get_idx2word()
documents = [' '.join([idx2word[idx] for idx in x_train[1:]]) for x_train in X_train] + \
            [' '.join([idx2word[idx] for idx in x_test[1:]]) for x_test in X_test]

tfidf = TfidfVectorizer(stop_words = 'english',
                        max_df = .1,
                        max_features = 5000)
tfidf_repr = tfidf.fit_transform(documents)

lda = LatentDirichletAllocation(n_components = N_TOPICS,
                                learning_method = 'batch',
                                max_iter = 100,
                                evaluate_every = 1,
                                verbose = 1)
lda.fit(tfidf_repr)

for topic_idx, term_vals in enumerate(lda.components_):
  message = "Topic #{}: ".format(topic_idx)
  message += " ".join([tfidf.get_feature_names()[i]
                       for i in term_vals.argsort()[:-MAX_TERMS-1:-1]])
  print(message)

iteration: 1 of max_iter: 100, perplexity: 8939.3527
iteration: 2 of max_iter: 100, perplexity: 8595.3923
iteration: 3 of max_iter: 100, perplexity: 8213.2271
iteration: 4 of max_iter: 100, perplexity: 7957.1807
iteration: 5 of max_iter: 100, perplexity: 7772.2066
iteration: 6 of max_iter: 100, perplexity: 7623.4844
iteration: 7 of max_iter: 100, perplexity: 7500.7866
iteration: 8 of max_iter: 100, perplexity: 7405.2129
iteration: 9 of max_iter: 100, perplexity: 7338.7826
iteration: 10 of max_iter: 100, perplexity: 7292.9838
iteration: 11 of max_iter: 100, perplexity: 7258.4895
iteration: 12 of max_iter: 100, perplexity: 7228.7228
iteration: 13 of max_iter: 100, perplexity: 7200.3117
iteration: 14 of max_iter: 100, perplexity: 7173.6886
iteration: 15 of max_iter: 100, perplexity: 7147.6127
iteration: 16 of max_iter: 100, perplexity: 7123.2638
iteration: 17 of max_iter: 100, perplexity: 7100.2275
iteration: 18 of max_iter: 100, perplexity: 7077.0315
iteration: 19 of max_iter: 100, perpl

In [6]:
import pyLDAvis
import pyLDAvis.sklearn

In [7]:
pyLDAvis.save_html(pyLDAvis.sklearn.prepare(lda, tfidf_repr, tfidf), 'lda.html')